# Strassenのアルゴリズム

N次の正方行列の積は、結果の行列の一要素の計算にN回の積とN-1≒N回の加算が必要で、  
要素数はNxNあるので全体で$O(N^3)$の計算量になるが、小行列に分割統治して計算するとほんの少しだけ計算量が少なく済む。  
大きなNに対して$O(N^{log_2{7}}) \simeq O(N^{2.8})$ で計算できる。詳しくは[奥村本](https://www.amazon.co.jp/dp/B07CG4RMT5)、[Wikipedia](https://en.wikipedia.org/wiki/Strassen_algorithm)。    
Pythonでそのままやるとnumpy.dot()の方が遥かに早い。残念。

In [1]:
import numpy as np

# そのまま実装
def mulNaive(A,B):
    size = A.shape[0]
    C = np.empty((size,size))
    for j in range(size):
        for i in range(size):
            tmp = 0
            for k in range(size):
                tmp += A[i,k] * B[k,j]
            C[i,j] = tmp
    return C
            
# Strassenのアルゴリズム
def mulStrassen(A,B):
    assert(A.shape == B.shape)
    shape = A.shape
    if shape[0] == 1 :
        return A[0,0]*B[0,0]
    #
    half = int(shape[0]/2)
    size = shape[0]
    A11 = A[0:half,    0:half]
    A12 = A[0:half,    half:size]
    A21 = A[half:size, 0:half]
    A22 = A[half:size, half:size]
    B11 = B[0:half,    0:half]
    B12 = B[0:half,    half:size]
    B21 = B[half:size, 0:half]
    B22 = B[half:size, half:size]
    #
    M1  = mulStrassen(A11+A22, B11+B22)
    M2  = mulStrassen(A21+A22, B11)
    M3  = mulStrassen(A11,B12-B22)
    M4  = mulStrassen(A22,B21-B11)
    M5  = mulStrassen(A11+A12,B22)
    M6  = mulStrassen(A21-A11,B11+B12)
    M7  = mulStrassen(A12-A22,B21+B22)
    #
    C11 = M1 + M4 - M5 + M7
    C12 = M3 + M5
    C21 = M2 + M4
    C22 = M1 + M3 - M2 + M6
    C = np.empty((size,size))
    C[0:half,    0:half]    = C11
    C[half:size, 0:half]    = C21
    C[0:half,    half:size] = C12
    C[half:size, half:size] = C22
    return C

# テスト
A = np.random.rand(64,64)
B = np.random.rand(64,64)
C0 = np.dot(A,B)
C1 = mulNaive(A,B)
C2 = mulStrassen(A,B)
[np.allclose(C0,C1),np.allclose(C0,C2)]

[True, True]

In [2]:
%%timeit
C0 = np.dot(A,B)

11.1 µs ± 414 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [3]:
%%timeit
C1 = mulNaive(A,B)

98.7 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%%timeit
C2 = mulStrassen(A,B)

336 ms ± 15.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
